### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.


In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

####  Add required Java Client Imports

These are some of the Aerospike Java Client imports needed to start developing our Application interactively.  We will add others, as needed, as we develop our solution.

In [4]:
//Require Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


#### Connect to the Aerospike Server
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [5]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


# Accessing a record on the Aerospike Server
We build the Key object in Java
We need the namespace and the record digest to find the record on the server.
The record digest is computed by the client library using application provided key (integer, string or byte-array) and the set name. If not in a set, use null for set name.

<img src="./graphics/RecordKey.png"
     alt="Record Digest"
     style="float: left; margin-right: 10px;"
     width="600"
     height="400"/>

In [6]:
Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);  //Will show the 20 byte digest

Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1


#### Test Record Data Model

![fig_DM_1](./graphics/Fraud_DM_1.png)

In [7]:
//Insert a record with two bins, b1 with string v1 and b2 with integer 2 as data.

WritePolicy wPolicy = new WritePolicy();

Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(28));

client.put(wPolicy, key, b1, b2);

In [8]:
//Read the record that we just inserted

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Read back the record.
Record values are:
(gen:1),(exp:450845426),(bins:(b1:val1),(b2:28))


## Record Data Model
While Aerospike limits bin names to 15 characters, shorter the better.

### Record Data Model:
* acct_id (Primary Key):
 *  ovr (override bin, boolean)
 *  lastamnt (last successful withdrawl_amount bin, integer, cents) 
 *  payee (payee bin, string)  
 *  bal (current_balance bin, integer, cents) 
 *  profile (user_profile_json bin, string, username) 
 
 ![fig_DM_2](./graphics/Fraud_DM_2.png)


#### New  _acct_id_ record creation

**Exercise** Write code to insert one record in namespace: **test**, setname: **accts**, key = **1** integer value (int acct_id=1). 
Add the bins described above with ovr=true, lastamnt=-10000, payee="SELF", bal=10000. 


In [9]:
// New acct_id record creation
// Lets create a test record with this data structure

int iAcct_id = 1;  //Acct_id = 1 to 1000.
boolean boolOvr = true;
int iWdAmount = -10000;
String sPayee = "SELF";
int iBal = 10000;

Key user = new Key("test", "accts", iAcct_id);  

Bin bOvr = new Bin("ovr", Value.get(boolOvr));
Bin bAmnt = new Bin("lastamnt", Value.get(iWdAmount));
Bin bPayee = new Bin("payee", Value.get(sPayee));
Bin bBal = new Bin("bal", Value.get(iBal));
Bin bProfile = new Bin("profile", Value.get("user"+iAcct_id));

WritePolicy wPolicy = new WritePolicy();
client.put(wPolicy, user, bOvr, bAmnt, bPayee, bBal, bProfile);



In [10]:
//Check 
System.out.println( client.get(null, user) );

(gen:1),(exp:450845426),(bins:(ovr:true),(lastamnt:-10000),(payee:SELF),(bal:10000),(profile:user1))


## Using OperationExpression write()



### Set all bins to null using Expressions, should delete the record.

In [11]:
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;

//int iAcct_id = 1;  //Acct_id = 1 to 1000.
//boolean boolOvr = false;
//int iWdAmount = 8000;
//String sPayee = "VISA";

//Key user = new Key("test", "accts", iAcct_id); 

//Set bins to null to delete the record
Expression balExp = Exp.build(Exp.nil());

WritePolicy wPolicy = new WritePolicy();
record = client.operate( wPolicy, user,  
          ExpOperation.write("lastamnt", balExp, ExpWriteFlags.ALLOW_DELETE),
          ExpOperation.write("payee", balExp, ExpWriteFlags.ALLOW_DELETE),
          ExpOperation.write("ovr", balExp, ExpWriteFlags.ALLOW_DELETE),              
          ExpOperation.write("profile", balExp, ExpWriteFlags.ALLOW_DELETE),              
          ExpOperation.write("bal", balExp, ExpWriteFlags.ALLOW_DELETE)  
         );


In [12]:
//Check 
System.out.println( client.get(null, user) );

null
